In [ ]:
import pandas as pd
import requests
import time
import pendulum
from sqlalchemy import create_engine
from dbconfig import MY_DWH
# подключаем возможность отключения проверки сертификата
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

# Подключение к базе данных
host, port, user, password, database = MY_DWH()
engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}")

# Функция для обработки JSON и преобразования в DataFrame
def parse_json_to_dataframe(data):
    parsed_data = {}

    # Обработка JSON по группам
    for section in data['data']:
        group_name = section[0]  # Название группы (например, "Моточасы", "Канал 1")
        for item in section[1]:
            key, value = item  # Параметр и его значение
            full_key = f"{group_name} - {key}"  # Уникальное имя колонки
            parsed_data[full_key] = value
    
    # Преобразуем данные в DataFrame с одной строкой
    df = pd.DataFrame([parsed_data])
    return df

# URL и заголовки для API-запроса
url = 'https://cnc.kovalev.team/get/3'
headers = {
    'Authorization': 'Bearer'  
}

# Бесконечный цикл для обработки JSON и записи в базу данных
while True:
    try:
        # Запрос данных с сервера
        response = requests.get(url, headers=headers, verify=False)
        
        # Проверяем статус ответа
        if response.status_code == 200:
            data = response.json()  # Получаем данные в формате JSON
            
            # Парсим JSON и преобразуем его в DataFrame
            parsed_data = parse_json_to_dataframe(data)
            current_time = pendulum.now(tz='Europe/Moscow')
            parsed_data['Дата'] = current_time.to_date_string()
            parsed_data['Время'] = current_time.to_time_string()
                        
            # Записываем данные в базу данных
            parsed_data.to_sql('ids3_table_dates', engine, schema='public', if_exists='append', index=False)
            
            # Выводим результат (для проверки)
            print(f"Новые данные успешно добавлены в таблицу ids3_table ({parsed_data.shape[0]} записей).")
        else:
            print(f"Ошибка: {response.status_code}, {response.text}")
    
    except Exception as e:
        print(f"Ошибка при обработке данных: {e}")
    
    # Пауза перед следующим запросом
    time.sleep(1)  # Интервал между запросами (в секундах)


Ошибка при обработке данных: HTTPSConnectionPool(host='cnc.kovalev.team', port=443): Max retries exceeded with url: /get/3 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000220870E3B20>: Failed to resolve 'cnc.kovalev.team' ([Errno 11001] getaddrinfo failed)"))
Ошибка при обработке данных: HTTPSConnectionPool(host='cnc.kovalev.team', port=443): Max retries exceeded with url: /get/3 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000220871383D0>: Failed to resolve 'cnc.kovalev.team' ([Errno 11001] getaddrinfo failed)"))
Ошибка при обработке данных: HTTPSConnectionPool(host='cnc.kovalev.team', port=443): Max retries exceeded with url: /get/3 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000022087138C10>: Failed to resolve 'cnc.kovalev.team' ([Errno 11001] getaddrinfo failed)"))
Ошибка при обработке данных: HTTPSConnectionPool(host='cnc.kovalev.team', port=443): Max retries exceeded wit

KeyboardInterrupt: 